In [1]:
# explorations
'''
https://huggingface.co/osanseviero/full-sentence-distillroberta2 - can be used to select final set of sentences
'''

'\nhttps://huggingface.co/osanseviero/full-sentence-distillroberta2 - can be used to select final set of sentences\n'

In [2]:
import os
import sys
import argparse
import torch
import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive
from openie import StanfordOpenIE
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader
import gensim
from transformers import pipeline
import csv
import contractions
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import MultiTagger
import re
from operator import itemgetter
import math


In [3]:
distilBertUnmasker = pipeline('fill-mask', model='distilbert-base-uncased')
nerTagger = pipeline("ner", grouped_entities=True)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)
/Users/bidishadasbaksi/opt/anaconda3/lib/python3.8/site-packages/transformers/pipelines/token_classification.py:128: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


## Read File

In [4]:
with open("dialogues_train.txt","r") as f:
    daily_dialog_list = f.read().split("\n")

In [5]:
conv_list = list()

### a. To append all parts of multi turn conversation

In [117]:

for conv in daily_dialog_list:
    s_l = conv[:len(conv)-len(" __eou__ ")].split(" __eou__ ")
    i=0
    while( i < (len(s_l)-1)):
        tup = (s_l[i],s_l[i+1])
        i=i+2
        conv_list.append(tup)

In [18]:
conv_list

[('Say , Jim , how about going for a few beers after dinner ?',
  'You know that is tempting but is really not good for our fitness .'),
 ('Can you do push-ups ?',
  "Of course I can . It's a piece of cake ! Believe it or not , I can do 30 push-ups a minute ."),
 ('Can you study with the radio on ?', 'No , I listen to background music .'),
 ('Are you all right ?',
  'I will be all right soon . I was terrified when I watched them fall from the wire .'),
 ('Hey John , nice skates . Are they new ?',
  'Yeah , I just got them . I started playing ice hockey in a community league . So , I finally got myself new skates .'),
 ('Hey Lydia , what are you reading ?',
  'I ’ m looking at my horoscope for this month ! My outlook is very positive . It says that I should take a vacation to someplace exotic , and that I will have a passionate summer fling !'),
 ('Frank ’ s getting married , do you believe this ?', 'Is he really ?'),
 ('I hear you bought a new house in the northern suburbs .',
  'That 

### b. To append only 2 turns

In [6]:

for conv in daily_dialog_list:
    s_l = conv[:len(conv)-len(" __eou__ ")].split(" __eou__ ")
    if(s_l[0] and s_l[1]):
        tup = (s_l[0],s_l[1])
    conv_list.append(tup)


In [7]:
## Verify length
len(conv_list)

11119

## Data Preprocessing and clean up

In [8]:

conv_list =[(contractions.fix(tup[0]),contractions.fix(tup[1])) for tup in conv_list]
               
               
               

### a. Verify Data Structures

In [9]:
conv_list[:100]

[('Say , Jim , how about going for a few beers after dinner ?',
  'You know that is tempting but is really not good for our fitness .'),
 ('Can you do push-ups ?',
  'Of course I can . it is a piece of cake ! Believe it or not , I can do 30 push-ups a minute .'),
 ('Can you study with the radio on ?', 'No , I listen to background music .'),
 ('Are you all right ?',
  'I will be all right soon . I was terrified when I watched them fall from the wire .'),
 ('Hey John , nice skates . Are they new ?',
  'Yeah , I just got them . I started playing ice hockey in a community league . So , I finally got myself new skates .'),
 ('Hey Lydia , what are you reading ?',
  'I ’ m looking at my horoscope for this month ! My outlook is very positive . It says that I should take a vacation to someplace exotic , and that I will have a passionate summer fling !'),
 ('Frank ’ s getting married , do you believe this ?', 'Is he really ?'),
 ('I hear you bought a new house in the northern suburbs .',
  'That

## Utility Functions

### Perplexity Scores

In [10]:
import math
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel
model = OpenAIGPTLMHeadModel.from_pretrained('openai-gpt')
model.eval()
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
def score(sentence):
    tokenize_input = tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    loss = model(tensor_input, lm_labels=tensor_input)
    return math.exp(loss)

In [187]:
score('I lost wallet, I dont know')

272.97357619337527

In [12]:
# utils

def getWordnetSynonymAndAntonynm(word):
    syn = list()
    ant = list()
    for synset in wordnet.synsets(word):
       for lemma in synset.lemmas():
          syn.append(lemma.name())    #add the synonyms
          if lemma.antonyms():    #When antonyms are available, add them into the list
              ant.append(lemma.antonyms()[0].name())
    return set(syn), set(ant)

def removeStopWords(string):
    filtered_words = [word for word in string.split() if word not in stopwords.words('english')]
    return filtered_words

posTagger = SequenceTagger.load("flair/pos-english")
def getPosTags(sentence):
    msentence = Sentence(sentence)

    # predict NER tags
    posTagger.predict(msentence)
    
    tags = [tag.labels[0].value for tag in msentence.get_spans('pos')]
    return list(zip(sentence.split(), tags))


2021-11-25 03:06:32,474 loading file /Users/bidishadasbaksi/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63


In [13]:
# https://huggingface.co/mrm8488/t5-base-finetuned-common_gen?text=tree+plant+ground+hole+dig
# https://github.com/INK-USC/CommonGen

from transformers import AutoModelWithLMHead, AutoTokenizer

commongen_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")
commongen_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-common_gen")

def commongen_sentence(words, max_length=32):
  input_text = words
  features = commongen_tokenizer([input_text], return_tensors='pt')
  output = commongen_model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return commongen_tokenizer.decode(output[0], skip_special_tokens=True)

# test model to verify functionality
words = "tree plant ground hole dig"
commongen_sentence(words)

# load comet - atomic and conceptnet models

# atomic

atomic_model_file = 'pretrained_models/atomic_pretrained_model.pickle'
atomic_opt, atomic_state_dict = interactive.load_model_file(atomic_model_file)
atomic_data_loader, atomic_text_encoder = interactive.load_data("atomic", atomic_opt)
atomic_n_ctx = atomic_data_loader.max_event + atomic_data_loader.max_effect
atomic_n_vocab = len(atomic_text_encoder.encoder) + atomic_n_ctx
atomicModel = interactive.make_model(atomic_opt, atomic_n_vocab, atomic_n_ctx, atomic_state_dict)

cfg.device = "cpu"
sampling_algorithm = 'topk-4'
category = "all"
atomic_sampler = interactive.set_sampler(atomic_opt, sampling_algorithm, atomic_data_loader)
atomic_all_categories = atomic_data_loader.categories

# returns results sorted by beam_loss
def getAtomicResult(input_event, category):
    sequence_all = {}
    sequence_all["event"] = input_event
    sequence_all["effect_type"] = category

    with torch.no_grad():

        batch = interactive.set_atomic_inputs(
            input_event, category, atomic_data_loader, atomic_text_encoder)

        sampling_result = atomic_sampler.generate_sequence(
            batch, atomicModel, atomic_data_loader, atomic_data_loader.max_event +
            data.atomic_data.num_delimiter_tokens["category"],
            atomic_data_loader.max_effect -
            data.atomic_data.num_delimiter_tokens["category"])
    
    results = zip(sampling_result["beam_losses"], sampling_result["beams"])
    sortedResults = sorted(results, key=lambda x: x[0])
    sortedResults = [result[1] for result in sortedResults]
    return sortedResults

def getAtomicResults(input_event):
    results = {}
    for category in atomic_all_categories:
        res = getAtomicResult(input_event, category)
        results[category] = res
    return results

# concept net

conceptnet_model_file = 'pretrained_models/conceptnet_pretrained_model.pickle'
conceptnet_opt, conceptnet_state_dict = interactive.load_model_file(conceptnet_model_file)

conceptnet_data_loader, conceptnet_text_encoder = interactive.load_data("conceptnet", conceptnet_opt)

conceptnet_n_ctx = conceptnet_data_loader.max_e1 + conceptnet_data_loader.max_e2 + conceptnet_data_loader.max_r
conceptnet_n_vocab = len(conceptnet_text_encoder.encoder) + conceptnet_n_ctx

conceptnet_model = interactive.make_model(conceptnet_opt, conceptnet_n_vocab, conceptnet_n_ctx, conceptnet_state_dict)

conceptnet_sampler = interactive.set_sampler(conceptnet_opt, sampling_algorithm, conceptnet_data_loader)
conceptnet_relation = "all"
conceptnet_all_relations = data.conceptnet_data.conceptnet_relations

# returns results sorted by beam_loss
def getConceptNetResult(input_sentence, relation, force=False):
    sequence_all = {}

    sequence_all["e1"] = input_sentence
    sequence_all["relation"] = relation

    with torch.no_grad():
        if conceptnet_data_loader.max_r != 1:
            relation_sequence = data.conceptnet_data.split_into_words[relation]
        else:
            relation_sequence = "<{}>".format(relation)

        batch, abort = interactive.set_conceptnet_inputs(
            input_sentence, relation_sequence, conceptnet_text_encoder,
            conceptnet_data_loader.max_e1, conceptnet_data_loader.max_r, force)

        if abort:
            return {}

        sampling_result = conceptnet_sampler.generate_sequence(
            batch, conceptnet_model, conceptnet_data_loader,
            conceptnet_data_loader.max_e1 + conceptnet_data_loader.max_r,
            conceptnet_data_loader.max_e2)
    results = zip(sampling_result["beam_losses"], sampling_result["beams"])
    sortedResults = sorted(results, key=lambda x: x[0])
    sortedResults = [result[1] for result in sortedResults]
    return sortedResults

def getConceptNetResults(input_sentence):
    results = {}
    for relation in conceptnet_all_relations:
        results[relation] = getConceptNetResult(input_sentence, relation)
    return results

/Users/bidishadasbaksi/opt/anaconda3/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:694: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Loading data from: data/atomic/processed/generation/categories_oEffect#oReact#oWant#xAttr#xEffect#xIntent#xNeed#xReact#xWant-maxe1_17-maxe2_35-maxr_1.pickle
52
Loading data from: data/conceptnet/processed/generation/rel_language-trainsize_100-devversion_12-maxe1_10-maxe2_15-maxr_5.pickle
31


In [14]:
posTagger = MultiTagger.load(['pos','ner'])

2021-11-25 03:06:51,801 --------------------------------------------------------------------------------
2021-11-25 03:06:51,802 The model key 'pos' now maps to 'https://huggingface.co/flair/pos-english' on the HuggingFace ModelHub
2021-11-25 03:06:51,803  - The most current version of the model is automatically downloaded from there.
2021-11-25 03:06:51,803  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/pos/en-pos-ontonotes-v0.5.pt)
2021-11-25 03:06:51,804 --------------------------------------------------------------------------------
2021-11-25 03:06:52,168 loading file /Users/bidishadasbaksi/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63
2021-11-25 03:06:53,251 --------------------------------------------------------------------------------
2021-11-25 03:06:53,251 The model key 'ner' now maps to 'https:/

### Similarity Scores

In [15]:

model_sim = SentenceTransformer('bert-base-nli-mean-tokens')

In [16]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
]

In [17]:
sentences = ['You know that is tempting but is really not good for our fitness .','i think i would go for a few beers after dinner.','if you want to take your fitness to the next level then you know where to go for a few beers after dinner.']

In [18]:
sentence_embeddings = model_sim.encode(sentences)

In [19]:
def get_sim_score(sentence_1, sentences):
    l = [sentence_1] + sentences
#     print(l)
    sentence_embeddings = model_sim.encode(l)
    cs_0 = cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:])
    l = list()
    i=0
    for v in cs_0[0]:
#         print(v)
        l.append([i,v])
        i=i+1
    return l
    
    

In [20]:
x = "Three years later, the coffin was still full of Jello."
y = [ "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."]

In [21]:
a = get_sim_score(x,y)

In [22]:
a 

[[0, 0.33088908], [1, 0.7219258], [2, 0.5548363]]

## Negative Sentence Generation

### a . Strategy 1

In [3]:
#### For Noun - appends not before the word , and checks the 'Desires' relation from ConceptNet for replacement

In [23]:
def remove_stopwords(sent_list):
    n_l =list()
    for s in sent_list:
        n_s = ' '.join([word for word in s.split() if word not in stopwords.words('english')])
        if(len(n_s)==0):
            continue
        n_l.append(n_s )
    return n_l

In [24]:
def get_pos_tags(s):
    sentence = Sentence(s)
    posTagger.predict(sentence)
    d = sentence.to_dict(tag_type='pos')
    return d

In [25]:
from collections import Counter

### Strategy 4 : Use Sim score too

In [1]:
def transform_sent_multi_output(d,c):
    mod_flag = False
    new_sent = [""]
    index = 0
    while index < len(d['entities']):
            tag = d['entities'][index]
            label = str(tag['labels'][0]).split(" ")[0]
            word_tok = tag['text']
            if( not(mod_flag) and label == 'JJ'):
#                 c.update(['JJ'])
                mod_flag = True
                flag = False
                for synset in wordnet.synsets(tag['text']):
                    for lemma in synset.lemmas():
                        if lemma.antonyms():
                            ant = lemma.antonyms()[0].name()
                            flag = True
                            break
                    if(flag):
                        break
                if(flag):
                    word_negate = ant
                else:
                    word_negate = "not "+tag['text']
                word_tok = word_negate
                new_sent_c = list()
                for s in new_sent:
                    new_sent_c.append(s+" "+word_negate)
#                 new_sent = new_sent+ " "+word_tok
                new_sent = new_sent_c
                index =index + 1
            elif(not(mod_flag) and label == 'NN'):
                mod_flag = True
                word_negate = tag['text']
                infs = getConceptNetResult(word_negate,'Desires')
                infs_m = min(len(infs),3)
                infs = infs[0:infs_m]
                new_sent_c = list()
                for s in new_sent:
                    for inf in infs:
                        new_text = "not "+ inf
                        word_tok = new_text
                        new_sent_c.append(s+" "+word_tok)
#                 new_sent = new_sent+ " "+word_tok
                new_sent = new_sent_c
#                 new_sent = new_sent + " "+word_tok
                index =index + 1
            elif(not(mod_flag) and len(label) >=2 and label[0:2] == 'VB'):
                mod_flag = True
                min_ind = max(0,index-3)
                max_ind = min(len(d['entities'])-1,index+3)
                vb_context = ""
                min_cont = ""
                for j in range(min_ind,max_ind+1):
                    vb_context += " "+ d['entities'][j]['text']
                    if(j<index):
                        min_cont += d['entities'][j]['text']
                infs = getConceptNetResult(vb_context,'HasLastSubevent')
                infs_m = min(len(infs),3)
                infs = infs[0:infs_m]
                new_sent_c = list()
                for s in new_sent:
                    for inf in infs:
                        word_tok = inf +" then "+ vb_context
                        ns = s.replace(min_cont,"")
                        new_sent_c.append(ns+" "+word_tok)
                new_sent = new_sent_c      
#                 new_sent += " "+word_tok
                index =index + 3
            else:
                index +=1
                new_sent_c = list()
                for s in new_sent:
                    new_sent_c.append(s+" "+word_tok)
                new_sent = new_sent_c
#     print("new sent : ",new_sent)
    return (new_sent,mod_flag)

In [2]:
mod_out = list()

In [ ]:
for i in range(8000,9000):
    response = conv_list[i][1].strip()
    if(response[len(response)-1]=='.'):
         response = response[:-1]
    print("Ind : ",i," Time : ",datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    response_list = response.split('.')
    com_sense = ""
    new_res = ""
    com_flag = False
    prev_sent = remove_stopwords([conv_list[i][0]])[0]
    resp_list_wo_stopwords = response_list[:]
#     print("resp_list_wo_stopwords : ",resp_list_wo_stopwords)
    c = Counter()
    com_sent_ap_c=[""]
    for resp in resp_list_wo_stopwords:
        d = get_pos_tags(resp)
        (mod_sent_list,mod_flag) = transform_sent_multi_output(d,c)
        com_sent_ap = list()
        for cs in com_sent_ap_c:
            for ns in mod_sent_list:
                com_sent_ap.append(cs+" "+ns+ ".")
        com_sent_ap_c = com_sent_ap
    com_flag = com_flag or mod_flag
    if(com_flag):
        print("Actual Tuple : ",conv_list[i])   
        print("Malformed Sentences : ",com_sent_ap_c)
        print("Commonsense Output")
        com_resp = list()
        len_to_iter = min(len(com_sent_ap_c),10)
        strategy1 = False
        if(strategy1):
            for i1,s in enumerate(com_sent_ap_c):
                cr = commongen_sentence(s,50)
    #             print(cr)
    #             print(score(cr))
                com_resp.append([cr,score(cr)])
    #             print(i1," ",cr," ",score(cr))
        else:
            window_size =2
            for ind,sent in enumerate(com_sent_ap_c[:len_to_iter]):
                if(len(sent.strip()) == 0):
                    continue
                com_sent_ap_c_l = [prev_sent] + sent.split('.')[:-1]
                i2=1
                while(i2<(len(com_sent_ap_c_l))):
                    s=""
                    cr_l = ""
                    j=i2
                    while(j>=i2-window_size and j >= 0 ):
                        s =  com_sent_ap_c_l[j] + " "+s 
                        j=j-1
                    cr = commongen_sentence(s,50)
                    cr_l  = cr_l +cr+"."
                    i2+=1
                com_resp.append([cr_l,score(cr_l)])
                print("Output ",ind," ",cr_l," ",score(cr_l),get_sim_score(conv_list[i][0],[cr_l])[0][1])
        com_resp_l = [w[0] for w in com_resp]
        sim_score = get_sim_score(conv_list[i][0],com_resp_l)
        sim_score_sort = sorted(sim_score,key=itemgetter(1),reverse=True)
        com_resp_new = list()
        for w3 in range(min(len(sim_score_sort),10)):
            com_resp_new.append(com_resp[sim_score_sort[w3][0]]+[sim_score_sort[w3][0]]+[sim_score_sort[w3][1]])
        com_resp_perp_sort = sorted(com_resp_new,key=itemgetter(1))
        print(conv_list[i][0],"|",com_resp_perp_sort[0][0],com_resp_perp_sort[0][1],com_resp_perp_sort[0][2],com_resp_perp_sort[0][3])
        mod_out.append([conv_list[i][0],com_resp_perp_sort[0][0],com_resp_perp_sort[0][1],com_sent_ap_c[com_resp_perp_sort[0][2]]])
        file_save_iter =10
        if(count_print%file_save_iter ==0 ):
            with open("output_5.txt","a") as f , open("output_malformed__5.txt","a") as f2:
                for e_i in range(watermark,len(mod_out)):
                    element = mod_out[e_i]
                    try:
                        f.write(element[0]+" __eou__ "+element[1]+"\n")
                        f2.write(element[0]+"\t | \t"+element[1]+ "\t | \t"+element[3]+'\t | \t'+str(element[2])+"\n")
                    except:
                        print("Exception occured. Moving on to next one.")
                watermark = len(mod_out)
            f.close()
            f2.close()
        count_print +=1
    
    

Ind :  10039  Time :  26/11/2021 17:20:34
Really ? How much is it ? Oh , it is 42 yuan ! How expensive !
['Really ? How much is it ? Oh , it is 42 yuan ! How expensive !']
 resp : Really ? How much is it ? Oh , it is 42 yuan ! How expensive !
Mod list :  [' Really ? How little is it ? Oh , it is 42 yuan ! How expensive !']
Actual Tuple :  ('Mary , come here . I have found the book our teacher recommended us to buy last week .', 'Really ? How much is it ? Oh , it is 42 yuan ! How expensive !')
Malformed Sentences :  ['  Really ? How little is it ? Oh , it is 42 yuan ! How expensive !.']
Commonsense Output
 com_sent_ap_c_l  ['  Really ? How little is it ? Oh , it is 42 yuan ! How expensive !', '']
Output  0   i really gotta buy this! how much is it?.   45.21069420508416 0.29046357
Mary , come here . I have found the book our teacher recommended us to buy last week . | i really gotta buy this! how much is it?. 45.21069420508416 0 0.29046357
Ind :  10040  Time :  26/11/2021 17:20:43
Hello 

Output  2   i am doing this for fun and fitness..   286.12364585655945 0.40671194
I hear you jog everyday . | i am doing this for fun and fitness.. 286.12364585655945 1 0.4067119
Ind :  10043  Time :  26/11/2021 17:21:24
Pete ! Nice to see you Well , on top the norm , you know , wife and kids and work , I ’ Ve actually gotten into doing some trading 
['Pete ! Nice to see you Well , on top the norm , you know , wife and kids and work , I ’ Ve actually gotten into doing some trading ']
 resp : Pete ! Nice to see you Well , on top the norm , you know , wife and kids and work , I ’ Ve actually gotten into doing some trading 
Mod list :  [' Pete ! nasty to see you Well , on top the norm , you know , wife and kids and work , I ’ Ve actually gotten into doing some trading']
Actual Tuple :  ('Hey , John ! I haven ’ t seen you in ages ! What ’ s new ? What have you been up to ?', 'Pete ! Nice to see you Well , on top the norm , you know , wife and kids and work , I ’ Ve actually gotten into doi

 resp :  Shanghai and Beijing are thousands of miles away 
Mod list :  [' Shanghai and Beijing reach beijing then  Shanghai and Beijing are thousands of miles miles away', ' Shanghai and Beijing cross ocean then  Shanghai and Beijing are thousands of miles miles away', ' Shanghai and Beijing visit them then  Shanghai and Beijing are thousands of miles miles away']
 resp :  How about bullet train ? Faster and relatively cheap , I think 
Mod list :  [' How about not kill train ? Faster and relatively cheap , I think', ' How about not burn train ? Faster and relatively cheap , I think', ' How about not fire train ? Faster and relatively cheap , I think']
Actual Tuple :  ('Mike , shall we book two tickets online ?', 'Fine . Shanghai and Beijing are thousands of miles away . How about bullet train ? Faster and relatively cheap , I think .')
Malformed Sentences :  ['  coarse.  Shanghai and Beijing reach beijing then  Shanghai and Beijing are thousands of miles miles away.  How about not kill

Output  2   if you are thinking about cleaning up your mess then it is about time you get it sorted out..   32.145233977949026 0.24146128
Here comes the bus . | if you are thinking about cleaning up your mess then it is about time you get it sorted out.. 32.145233977949026 2 0.2414612
Ind :  10049  Time :  26/11/2021 17:23:02
Okay , let me take a look 
['Okay , let me take a look ']
 resp : Okay , let me take a look 
Mod list :  [' Okay , drink then  Okay , let me take a a look', ' Okay , swallow then  Okay , let me take a a look', ' Okay , take this then  Okay , let me take a a look']
Actual Tuple :  ('It has a receiver , a CD player , a double cassette deck , and a turntable .', 'Okay , let me take a look .')
Malformed Sentences :  ['  Okay , drink then  Okay , let me take a a look.', '  Okay , swallow then  Okay , let me take a a look.', '  Okay , take this then  Okay , let me take a a look.']
Commonsense Output
 com_sent_ap_c_l  ['  Okay , drink then  Okay , let me take a a look', 

Output  5   besides, i have worked at the company for four years and got a lot of practical experience..   26.92537429185553 0.18426955
 com_sent_ap_c_l  ['   pass it then  I have four years study study in the Department of Mechanical Engineering and it has given me a solid theory foundation', '  Moreover , I open door then  Moreover , I have worked at CAB CAB Company for 4 years and got a lot of practical experience', '']
Output  6   Moreover, I have worked at the company for 4 years and got a lot of experience in the field of mechanical engineering..   20.131662514720297 0.18195441
 com_sent_ap_c_l  ['   pass it then  I have four years study study in the Department of Mechanical Engineering and it has given me a solid theory foundation', '  Moreover , I pay fare then  Moreover , I have worked at CAB CAB Company for 4 years and got a lot of practical experience', '']
Output  7   besides, I have worked at the CAB Company for 4 years and got a lot of practical experience..   44.17011838

Output  0   i have no idea about the temperature of the water then i am afraid it might be a little cold..   29.221475968405805 0.20492391
 com_sent_ap_c_l  ['   watch tv then  You see this is the the first time for me to use the swimming pool', '   turn on tv then  I have got no idea idea about the water temperature of the pool', '   die then  I am afraid it might might be a little cold', '']
Output  1   i have no idea about the temperature of the water then i am afraid it might be a little cold..   29.221475968405805 0.20492391
 com_sent_ap_c_l  ['   watch tv then  You see this is the the first time for me to use the swimming pool', '   turn on tv then  I have got no idea idea about the water temperature of the pool', '   be recycle then  I am afraid it might might be a little cold', '']
Output  2   i have no idea about the temperature of the water in the pool then i am afraid it might be a little cold..   25.022333412696877 0.3254911
 com_sent_ap_c_l  ['   watch tv then  You see thi

Output  0   how's your family??.   82.38490476175845 0.36673304
 com_sent_ap_c_l  ['  unhappy to see you-too-Mike', "  How ' visit them then  How ' s your family ? ?", '']
Output  1   how's your family??.   82.38490476175845 0.36673304
 com_sent_ap_c_l  ['  unhappy to see you-too-Mike', "  How ' see your family then  How ' s your family ? ?", '']
Output  2   how's your family??.   82.38490476175845 0.36673304
It was good to see you here , John . | how's your family??. 82.38490476175845 0 0.36673304
Ind :  10056  Time :  26/11/2021 17:25:02
I would like to have my coat washed 
['I would like to have my coat washed ']
 resp : I would like to have my coat washed 
Mod list :  [' I would eat then  I would like to have my my coat washed', ' I would say goodbye then  I would like to have my my coat washed', ' I would wash then  I would like to have my my coat washed']
Actual Tuple :  ('Good evening , sir . What can I do for you ?', 'I would like to have my coat washed .')
Malformed Sentences 

Output  0   Can I just smoke in the aisle then sit down?.   132.78983882843468 0.17502156
 com_sent_ap_c_l  ['  Yes , say goodbye then  Yes , thank you', '  I just clean then  I just want to have a a smoke', '  Can I exhale then  Can I smoke in the aisle aisle ?', '']
Output  1   Can I just exhale then have a a smoke in the aisle?.   224.97985314744372 0.25308806
 com_sent_ap_c_l  ['  Yes , say goodbye then  Yes , thank you', '  I just clean then  I just want to have a a smoke', '  Can I cough then  Can I smoke in the aisle aisle ?', '']
Output  2   Can I just cough then have a smoke in the aisle?.   156.07257167832663 0.1944508
 com_sent_ap_c_l  ['  Yes , say goodbye then  Yes , thank you', '  I just smile then  I just want to have a a smoke', '  Can I sit down then  Can I smoke in the aisle aisle ?', '']
Output  3   Can I just sit down then have a a smoke in the aisle?.   132.1352182516973 0.21137652
 com_sent_ap_c_l  ['  Yes , say goodbye then  Yes , thank you', '  I just smile then

 resp : I will see if she is in 
Mod list :  [' I will  then  I will see if she is is in', ' I will eat then  I will see if she is is in', ' I will die then  I will see if she is is in']
Actual Tuple :  ('Hello , is Marie Ward there , please ?', 'I will see if she is in .')
Malformed Sentences :  ['  I will  then  I will see if she is is in.', '  I will eat then  I will see if she is is in.', '  I will die then  I will see if she is is in.']
Commonsense Output
 com_sent_ap_c_l  ['  I will  then  I will see if she is is in', '']
Output  0   if she is in a hospital then i will see if she is in a hospital ward.   17.51669891627483 0.4486954
 com_sent_ap_c_l  ['  I will eat then  I will see if she is is in', '']
Output  1   I will see if she is in the zoo then I will see if she is in the zoo.   14.644015078521893 0.31271768
 com_sent_ap_c_l  ['  I will die then  I will see if she is is in', '']
Output  2   if she is in a hospital then i will see if she is in a hospital..   22.2836340617015

In [174]:
f1.close()
f2.close()